In [ ]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train=torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test =torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size=32, shuffle=False, num_workers=2)


100%|██████████| 170M/170M [00:03<00:00, 43.8MB/s]


In [ ]:
image , label = train[0]

In [ ]:
image.size()

torch.Size([3, 32, 32])

In [ ]:
class_name = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = NeuralNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(30):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"loss : {running_loss/len(train_loader):.4f}")





loss : 1.5532
loss : 1.4137
loss : 1.3113
loss : 1.2215
loss : 1.1493
loss : 1.0888
loss : 1.0347
loss : 0.9873
loss : 0.9443
loss : 0.9027
loss : 0.8662
loss : 0.8304
loss : 0.7989
loss : 0.7687
loss : 0.7414
loss : 0.7136
loss : 0.6866
loss : 0.6592
loss : 0.6362
loss : 0.6122
loss : 0.5923
loss : 0.5666
loss : 0.5457
loss : 0.5284
loss : 0.5067
loss : 0.4851
loss : 0.4641
loss : 0.4463
loss : 0.4305
loss : 0.4088


In [ ]:
torch.save(net.state_dict(), 'trained_net.pth')

In [ ]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

<All keys matched successfully>

In [22]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 68.92%


In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = net.to(device)

new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = new_transform(image)
    image = image.unsqueeze(0).to(device)
    return image

image_path = ['example1.jpg','example2.jpg']
images = [predict_image(path) for path in image_path]

net.eval()
with torch.no_grad():
    for image in images:
        outputs = net(image)
        _, predicted = torch.max(outputs, 1)
        print(f"Predicted class: {class_name[predicted.item()]}")

Predicted class: plane
Predicted class: ship
